<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool37_Almudaya_Penetration_Testing_API_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pengujian Analisis Auto Respon API

In [8]:
import requests
from requests.exceptions import SSLError, ConnectionError
import ssl
import json

In [9]:
# Menggunakan JSONPlaceholder sebagai contoh
base_url = "https://jsonplaceholder.typicode.com/posts"

# Data untuk pengujian
payload = {
    "title": "foo",
    "body": "bar",
    "userId": 1
}

# Header untuk permintaan
headers = {
    "Content-Type": "application/json"
}

try:
    # 1. Mengirim permintaan POST dan mendapatkan respons
    response = requests.post(base_url, json=payload, headers=headers)

    # 2. Pengujian Status Kode
    print("Status Code Test")
    print("Status Code:", response.status_code)
    print("------")

    # 3. Pengujian Struktur Respons
    print("Response Structure Test")
    try:
        response_json = response.json()
        print("Response is valid JSON.")
        expected_keys = {"title", "body", "userId", "id"}
        if expected_keys.issubset(response_json.keys()):
            print("Response structure is valid.")
        else:
            print("Response structure is invalid.")
    except ValueError:
        print("Response is not valid JSON.")
    print("------")

    # 4. Pengujian Pesan Kesalahan
    print("Error Message Test")
    error_response = requests.post(base_url, json={}, headers=headers)
    print("Status Code:", error_response.status_code)
    if error_response.status_code >= 400:
        print("Error Message:", error_response.text)
        if "sensitive" not in error_response.text:
            print("Error message does not contain sensitive information.")
        else:
            print("Error message contains sensitive information.")
    print("------")

    # 5. Pengujian Penggunaan Header
    print("Response Headers Test")
    security_headers = ["Content-Security-Policy", "X-Content-Type-Options", "Strict-Transport-Security", "X-Frame-Options"]
    for header in security_headers:
        if header in response.headers:
            print(f"{header} is present: {response.headers[header]}")
        else:
            print(f"{header} is missing.")
    print("------")

    # 6. Pengujian Penggunaan SSL/TLS
    print("SSL/TLS Test")
    try:
        response = requests.get(base_url)
        if response.url.startswith("https://"):
            print("Connection is secure with SSL/TLS.")
        else:
            print("Connection is not secure.")
    except (SSLError, ConnectionError) as e:
        print(f"SSL/TLS Test failed: {e}")
    print("------")

    # 7. Pengujian Validasi Sertifikat SSL/TLS
    print("SSL/TLS Certificate Validation Test")
    try:
        response = requests.get(base_url, verify=True)
        print("SSL/TLS Certificate is valid.")
    except SSLError as e:
        print(f"SSL/TLS Certificate Validation failed: {e}")
    print("------")

    # 8. Pengujian Terhadap Serangan Man-in-the-Middle (MitM)
    print("Man-in-the-Middle (MitM) Attack Test")
    try:
        response = requests.get(base_url, verify=True)
        print("No MitM attack detected, SSL/TLS connection is secure.")
    except SSLError as e:
        print(f"MitM Attack Test failed: {e}")
    print("------")

    # 9. Pengujian Keamanan Input dan Parameter
    print("Input and Parameter Security Test")
    injection_payload = {
        "title": "'; DROP TABLE users; --",
        "body": "bar",
        "userId": 1
    }
    xss_payload = {
        "title": "<script>alert('XSS');</script>",
        "body": "bar",
        "userId": 1
    }
    large_payload = {
        "title": "A" * 10000,
        "body": "B" * 10000,
        "userId": 1
    }

    payloads = [injection_payload, xss_payload, large_payload]
    for i, payload in enumerate(payloads):
        response = requests.post(base_url, json=payload, headers=headers)
        print(f"Test {i+1} Status Code:", response.status_code)
        print(f"Test {i+1} Response Body:", response.json())
        print("------")

except Exception as e:
    print(f"An error occurred: {e}")

Status Code Test
Status Code: 201
------
Response Structure Test
Response is valid JSON.
Response structure is valid.
------
Error Message Test
Status Code: 201
------
Response Headers Test
Content-Security-Policy is missing.
X-Content-Type-Options is present: nosniff
Strict-Transport-Security is missing.
X-Frame-Options is missing.
------
SSL/TLS Test
Connection is secure with SSL/TLS.
------
SSL/TLS Certificate Validation Test
SSL/TLS Certificate is valid.
------
Man-in-the-Middle (MitM) Attack Test
No MitM attack detected, SSL/TLS connection is secure.
------
Input and Parameter Security Test
Test 1 Status Code: 201
Test 1 Response Body: {'title': "'; DROP TABLE users; --", 'body': 'bar', 'userId': 1, 'id': 101}
------
Test 2 Status Code: 201
Test 2 Response Body: {'title': "<script>alert('XSS');</script>", 'body': 'bar', 'userId': 1, 'id': 101}
------
Test 3 Status Code: 201
Test 3 Response Body: {'title': 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA